In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from keras import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.metrics import Recall, Precision
import keras.backend as K
from sklearn.preprocessing import OneHotEncoder

RANDOM_STATE = 123
VERBOSITY = 1


2023-04-11 11:53:14.797970: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 11:53:15.130786: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 11:53:15.134170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 11:53:18.038491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("task1.csv", index_col=0).sort_index()


In [3]:

y = df.pop("user_id")
y = y.apply(lambda x: 1 if x == 0 else 0)
y = pd.DataFrame(y)

cols_to_drop = ["time", "date", "sites"]
df.drop(columns=cols_to_drop)
X = df


def onehot_encode(X: pd.DataFrame) -> pd.DataFrame:
    encoder = OneHotEncoder()
    encoder.fit(X)
    np_array = encoder.transform(df).toarray()
    feat_columns = encoder.get_feature_names_out(df.columns)

    res_df = pd.DataFrame(np_array, columns=feat_columns)
    return res_df


print("Encoding...")

# X = pd.get_dummies(X)
X = onehot_encode(X)

print("Completed encoding")


Encoding...
Completed encoding


# New Section

In [4]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)



KeyboardInterrupt



In [ ]:

def my_f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val


def build_model(neurons_cnt=16, layers_cnt=1, dropout_prob=0.5):
    model = Sequential()

    model.add(Dense(neurons_cnt, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_prob))

    for i in range(layers_cnt):
        model.add(Dense(neurons_cnt, activation='relu'))
        model.add(Dropout(dropout_prob))

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=[
        my_f1_score,
        'accuracy',
        Precision(),
        Recall(),
    ]
)
    
    return model


model = build_model()



In [ ]:
# early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
# model.fit(X_train, y_train, validation_split=0.2, epochs=50, callbacks=[early_stopping])
# loss, accuracy = model.evaluate(X_test, y_test)

VALIDATION_RATIO = 0.2
BATCH_SIZE = 128
EPOCHS = 50

early_stopping = EarlyStopping(
    patience=5,
    restore_best_weights=True
)

print("model.fit")
model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_RATIO,
    verbose=VERBOSITY,
    callbacks=[early_stopping]
)

test_loss, test_f1, test_accuracy, test_precision, test_recall = model.evaluate(X_test, y_test)



In [ ]:
from sklearn.metrics import classification_report

# evaluate the model without fine tuning
y_pred = model.predict(X_test, verbose=VERBOSITY) > 0.5
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=build_model, verbose=VERBOSITY)

param_grid = {
    'neurons_cnt': [16, 32, 64],
    'layers_cnt': [1, 2, 3],
    'dropout_prob' : [0.0, 0.1, 0.2],
}

# Fine tune model
grid_search = GridSearchCV(model, param_grid=param_grid, cv=3, scoring='f1', verbose=VERBOSITY)
grid_result = grid_search.fit(X_train, y_train, verbose=2)

print(grid_result.best_score_)
print(grid_result.best_params_)


NameError: name 'grid_result' is not defined